Preamble
========

We have a database file (example.sqlite) in sqlite3 format with some tables and data. All tables have 'name' column and maybe some additional ones.

Data retrieval and modifications are done with sqlite3 module by issuing SQL statements.
For example, to get all data from TABLE1::

    import sqlite3
    conn = sqlite3.connect('example.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * from TABLE1')
    data = cursor.fetchall()   # will be a list with data.

instead of getting all data at once, you can use .fetchone() calls and named expressions::

    while row:=cursor.fetchone():
        print(row)

To get a row with specific name equal to some value::

    import sqlite3
    conn = sqlite3.connect('example.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * from presidents where name=:name', {name:'Yeltsin'})
    data = cursor.fetchall()  # will get all records with this name. You can also use .fetchone() to get one record.

in order to get record with first name (sorted alphabetically) use SQL expression `SELECT * from presidents order by name asc limit 1`
in order to get record after specified (sorted alphabetically) use SQL expression `SELECT * from presidents where name > :name order by name limit`.
To get amount of records in table TABLE1, use `select count(*) from TABLE1` query.


Please refer to this documents for more information about how to retrieve data from sqlite database:
DBAPI: https://www.python.org/dev/peps/pep-0249/
sqlite3 module: https://docs.python.org/3/library/sqlite3.html

Task
====

Write a wrapper class TableData for database table, that when initialized with database name and table acts as collection object (implements Collection protocol).
Assume all data has unique values in 'name' column.
So, if presidents = TableData(database_name='example.sqlite', table_name='presidents')

then
 -  `len(presidents)` will give current amount of rows in presidents table in database
 -  `presidents['Yeltsin']` should return single data row for president with name Yeltsin
 -  `'Yeltsin' in presidents` should return if president with same name exists in table
 -  object implements iteration protocol. i.e. you could use it in for loops::
       for president in presidents:
           print(president['name'])
 - all above mentioned calls should reflect most recent data. If data in table changed after you created collection instance, your calls should return updated data.

Avoid reading entire table into memory. When iterating through records, start reading the first record, then go to the next one, until records are exhausted.
When writing tests, it's not always neccessary to mock database calls completely. Use supplied example.sqlite file as database fixture file.

In [63]:
#https://docs.python.org/3/library/collections.abc.html
from collections.abc import Collection
class TableData(Collection):
    def __init__(self, database_name: str, table_name: str):
        self.database_name = database_name
        self.table_name = table_name
    
    def __contains__(self, item) -> bool:
        conn = sqlite3.connect('example.sqlite')
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM {self.table_name};")
        data = cursor.fetchall()
        for x in data:
            if item in x:
                return (item in x)
    
    def __len__(self) -> int:
        conn = sqlite3.connect('example.sqlite')
        cursor = conn.cursor()
        cursor.execute(f"SELECT count(*) FROM {self.table_name};")
        data = cursor.fetchall()
        return data[0][0]
    
    def __iter__(self):
        conn = sqlite3.connect('example.sqlite')
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM {self.table_name};")
        data = cursor.fetchall()
        for x in data:
            yield x
    
    def __getitem__(self, item):
        conn = sqlite3.connect('example.sqlite')
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM {self.table_name};")
        data = cursor.fetchall()
        for x in data:
            if item in x:
                return x
        

In [2]:
import sqlite3
conn = sqlite3.connect('example.sqlite')
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
data = cursor.fetchall()
for x in data:
    print(x)

('books',)
('presidents',)


In [64]:
tb = TableData('example.sqlite', 'presidents')
for x in tb:
    print(x)

('Yeltsin', 999, 'Russia')
('Trump', 1337, 'US')
('Big Man Tyrone', 101, 'Kekistan')


In [80]:
print("*Длина объекта* - " + str(len(tb)))
print("*Строка с президентом*")
print(tb['Big Man Tyrone'])
print("*Проверка на наличие президента* - " + str("Trump" in tb))
print("*Проверка итерации в цикле*")
for president in tb:
    print(president[0])

*Длина объекта* - 3
*Строка с президентом*
('Big Man Tyrone', 101, 'Kekistan')
*Проверка на наличие президента* - True
*Проверка итерации в цикле*
Yeltsin
Trump
Big Man Tyrone


In [85]:
tb2 = TableData('example.sqlite', 'books')
for x in tb2:
    print(x)
print("*Длина объекта*")
print(len(tb2))
print("*Строка с автором*")
print(tb2['Huxley'])
print("*Проверка на наличие книги*")
print("Brave New World" in tb2)
print("*Проверка итерации в цикле*")
for book in tb2:
    print(book[0])

('Farenheit 451', 'Bradbury')
('Brave New World', 'Huxley')
('1984', 'Orwell')
*Длина объекта*
3
*Строка с автором*
('Brave New World', 'Huxley')
*Проверка на наличие книги*
True
*Проверка итерации в цикле*
Farenheit 451
Brave New World
1984
